In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

print("🚀 Starting model training...")

# Paths
train_dir = 'resized_dataset_split/train'
val_dir = 'resized_dataset_split/val'
img_size = (224, 224)
batch_size = 32
epochs = 25

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Class weights
labels = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print("📊 Class weights:", class_weights)

# MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=2)
]

# Training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=callbacks
)

# Save model
model.save("mobilenetv2_balanced_model.h5")
print("✅ Model training complete and saved as mobilenetv2_balanced_model.h5")


🚀 Starting model training...
Found 269 images belonging to 3 classes.
Found 69 images belonging to 3 classes.
📊 Class weights: {0: 1.6011904761904763, 1: 0.5977777777777777, 2: 1.4232804232804233}


c:\Users\venkatesh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.3293 - loss: 1.4547 - val_accuracy: 0.4493 - val_loss: 1.0609 - learning_rate: 0.0010
Epoch 2/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 930ms/step - accuracy: 0.5390 - loss: 0.9129 - val_accuracy: 0.7246 - val_loss: 0.7109 - learning_rate: 0.0010
Epoch 3/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 946ms/step - accuracy: 0.6939 - loss: 0.7563 - val_accuracy: 0.7971 - val_loss: 0.5807 - learning_rate: 0.0010
Epoch 4/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7490 - loss: 0.5885 - val_accuracy: 0.8116 - val_loss: 0.5379 - learning_rate: 0.0010
Epoch 5/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7598 - loss: 0.4728 - val_accuracy: 0.8406 - val_loss: 0.4356 - learning_rate: 0.0010
Epoch 6/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.8064 - loss: 0.4682 - val_accuracy: 0.8696 - val_loss: 0.3995 - learning_rate: 0.0010
Epoch 7/25
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.8175 - loss: 0.4235 - val_accuracy: 0.8696 - va

✅ Model training complete and saved as mobilenetv2_balanced_model.h5
